In [12]:
#import libraries
from sklearn.datasets import load_boston # dataset
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.compat import lzip
import statsmodels.stats.api as sm
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
import statsmodels as sm
import statsmodels.formula.api as smf
from sklearn import linear_model

%matplotlib inline

In [13]:
# instantiate dataset and create DataFrame 
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
target = pd.DataFrame(boston.target, columns=["MEDV"])

In [14]:
# Basic exploratory analyses 
print("Rows,Columns:",df.shape)
df.describe()

Rows,Columns: (506, 13)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.593761,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.596783,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.647423,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [15]:
# split dataset into training and testing set
X_train, X_test, Y_train, Y_test = train_test_split(df, target, test_size=0.30)


In [16]:
# Fitting using stats model
model_1 = smf.OLS(Y_train,X_train).fit()
prediction_1 = model_1.predict(X_train) # predict y_train values
prediction_2 = model_1.predict(X_test)# predict y_test values

In [17]:
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.955
Model:                            OLS   Adj. R-squared:                  0.953
Method:                 Least Squares   F-statistic:                     557.1
Date:                Sun, 07 Jan 2018   Prob (F-statistic):          1.17e-220
Time:                        17:56:49   Log-Likelihood:                -1078.8
No. Observations:                 354   AIC:                             2184.
Df Residuals:                     341   BIC:                             2234.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.0956      0.043     -2.236      0.026      -0.180      -0.011
ZN             0.0554      0.017      3.192      0.002       0.021       0.089
INDUS          0.0056      0.082      0.068      0.946      -0.155       0.166
CHAS           1.9569      1.172      1.670      0.096      -0.348       4.262
NOX           -2.8633      4.068     -0.704      0.482     -10.865       5.138
RM             5.4115      0.389     13.909      0.000       4.646       6.177
AGE            0.0126      0.018      0.710      0.478      -0.022       0.047
DIS           -0.8650      0.239     -3.622      0.000      -1.335      -0.395
RAD            0.1944      0.081      2.388      0.018       0.034       0.355
TAX           -0.0103      0.005     -2.192      0.029      -0.019      -0.001
PTRATIO       -0.2925      0.139     -2.104      0.036      -0.566      -0.019
B              0.0158      0.003      4.577      0.000       0.009       0.023
LSTAT         -0.4700      0.063     -7.461      0.000      -0.594      -0.346
==============================================================================
Omnibus:                      134.653   Durbin-Watson:                   1.882
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              653.132
Skew:                           1.552   Prob(JB):                    1.49e-142
Kurtosis:                       8.886   Cond. No.                     8.28e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.28e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Looking at the Summary

In the summary we see some interesting results. The INDUS coefficient (proportion of non-retail business acres per town) is not nearly statistically significant at the 5% or 10% level. The same goes for the AGE and NOX variables.

There seems to be no autocorrelation (as expected as this is not a time-series problem).

Overall, the Adjusted R^2 score is great, but how about RMSE (root mean square error), lets have a loo

In [18]:
rmse_training = sqrt(mean_squared_error(Y_train, prediction_1))
print("RMSE for training data:", rmse_training)
rmse_test = sqrt(mean_squared_error(Y_test,prediction_2))
print("RMSE for test data:", rmse_test)

RMSE for training data: 5.095701134618519
RMSE for test data: 4.580486233041159
